# 모듈, 라이브러리 준비

In [1]:
  
# SKT Brain의 KoBERT, 학습 및 테스트 데이터셋만 따로 준비 = 카카오 브런치[text, label]
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
#!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup


     |████████████████████████████████| 47.3 MB 168 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595737 sha256=686dab94fe50b7f94764deef4bfbc454e99179cbb2d7d38402f15a689cceb330
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 769 kB 4.0 MB/s 
     |████████████████████████████████| 895 kB 88.6 MB/s 
     |████████████████████████████████| 3.0 MB 62.2 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ua7h60qg
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

# 환경 세팅

In [2]:
# GPU 사용 시
device = torch.device("cuda:0")
#device = torch.device("cpu")

# BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [3]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 준비

In [55]:
# 데이터셋 불러오기
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/데캡디/감성대화/sentiment_label4.csv')

# 해당 column만 추출
data.head()

,content,label
0,와 강원진짜 대박이네...단장 능력오지네,1
1,대리점들도망하게하는..,3
2,결혼식에 내가 좋아하는 가수가 축가를 불러주기로 했어! 너무 기뻐.,0
3,이번 동아리 엠티는 선배들 없이 우리 학번과 후배들만 간다고 해. 너무 신나.,0
4,쉽고 간단하게 설명해 주셔서 감사홧팅!,0


In [5]:
data = data.sample(frac=1)

In [6]:
data['label'].value_counts()

1    32171
2    18806
0    12767
3     4830
Name: label, dtype: int64

In [7]:
data_1 = data[data['label'] == 1]
data_2 = data[data['label'] == 2]
data_0 = data[data['label'] == 0]

In [56]:
data[data['label'] == 3]

,content,label
1,대리점들도망하게하는..,3
9,총맞겠네,3
16,음주 과속안했고 잘 달리고 있었으면 끼어들어서 사고유발한 흰 차가 제일 잘못한거지,3
17,택시기사님들이나 교육 잘 시키세요...,3
22,여기 경기도 군포인데해겁나 쨍쨍한데 눈부셔,3
...,...,...
68531,"밉상, 빨리 잡아들여야되는데",3
68535,턱받이하고 목욕시켜,3
68559,아이폰으로 보고있는 사람 손,3
68562,대한민국은 좌좀넘들만 없으면 참 훌륭한 나라이다.,3


In [8]:
 data[data['label'] == 0]

,content,label
62247,휴 정말 다행이지 뭐야.,0
38763,우리 가족들 모두 건강 검진했는데 건강해서 걱정도 없고 편안한 기분이야.,0
34519,멋쪄 멋쪄 다 멋쪄,0
23849,개인적으로 8:45 heaven 젤 좋았음 초명곡 ㄷㄷ,0
42221,어서 주말이 왔으면 좋겠어. 너무 기대된다.,0
...,...,...
574,우연히 연아양 경기모습을 보게되었어요....,0
24122,내일 정기검진을 받으러 가. 요즘 컨디션이 좋아서 결과가 괜찮을 것 같아.,0
2507,잊고 있었던 주식이 올라 너무 기쁘네.,0
24119,이제 월요일이 오히려 즐겁다고.,0


In [9]:
data[data['label'] == 1]

,content,label
12561,남편하고 사이가 안 좋아져서 애들 보기 미안해.,1
31084,자궁암 검사를 받았는데 아무래도 미심쩍어.,1
22928,점장님은 토요일 오전 09시에 끝나는거라던데어떤게 맞아요?,1
54843,아 진짜 지독한 녀언이 18대 대통련..,1
31326,검진 결과를 받았는데 만성질환이 더 심해졌다니 당황스럽네.,1
...,...,...
54580,그것땜에 어찌나 짜증났던지...,1
34126,생산관리 끝판왕 ㅠㅠ,1
53071,혹시 팀장님께서 아까 내가 한 말을 오해하셨을까 봐 걱정돼.,1
23647,오늘 시험을 봤는데 친구가 자기가 푼 답이 자꾸 정답이라고 우겨서 너무 짜증나.,1


In [10]:
data[data['label'] == 2]

,content,label
65858,서민들은 물가가 올라서 걱정이 태산인데...,2
11406,친척 모임에서 좋은 고등학교에 합격한 아이들만 예쁨 받는 것 같다는 생각에 실망했어.,2
16119,누나내가 좋아하는건 미안한일이잖아,2
35382,돈을 많이 못 쓰니 친구도 없고 그러네.,2
49712,건강이 점점 나빠져서 괴로워. 너무 괴로워서 눈물이 나.,2
...,...,...
54257,어떻게야할지 ㅠㅠㅠㅠㅠㅠ,2
23589,나를 괴롭히는 애들이 있는데 그 애들이 내 친한 친구의 물건을 훔치고는 나를 도둑으...,2
8998,매번 꼼꼼하게 완성해야만 하는 업무가 너무 힘들고 지쳐.,2
25086,나 오늘 엄청나게 충격받은 일이 있었어.,2


In [ ]:
data_1_sample = data_1[:12767]
data_2_sample = data_2[:12767]

In [ ]:
data = pd.concat([data_1_sample, data_2_sample, data_0])

In [ ]:
data = data.sample(frac=1)

In [ ]:
data['label'].value_counts()

2    12767
1    12767
0    12767
Name: label, dtype: int64

In [11]:
# 데이터 합치기
new_data = data

In [12]:
# label encoding
new_data.loc[(new_data['label'] == 0), 'label'] = 0  #발라드 => 0
new_data.loc[(new_data['label'] == 1), 'label'] = 1  #밤 => 0
new_data.loc[(new_data['label'] == 2), 'label'] = 2  #슬픈 => 0
new_data.loc[(new_data['label'] == 3), 'label'] = 3  #슬픈 => 0

# list형태로 데이터 저장 -> [content, label] 형태
data_list = []
for q, label in zip(new_data['content'], new_data['label'])  :
    data = []
    data.append(str(q)) # str타입으로 변환, 안 해주면 나중에 타입변환할 때 오류남
    data.append(str(label))

    data_list.append(data)

In [13]:
# 데이터 확인
print(data_list[0])
print(data_list[-1])

['휴 정말 다행이지 뭐야.', '0']
['2주전부터 이유없는 왕따?', '1']


In [14]:
# train, test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [15]:
# 데이터 개수 확인
print(len(dataset_train))
print(len(dataset_test))

51430
17144


# KoBERT 모델 준비

In [16]:
# KoBERT모델의 입력으로 들어갈 수 있는 형태가 되도록 변환해주는 class
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        # BERTSentenceTransform이라는 모듈 사용
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        # 타입 변환
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    # item 가져오기
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    # 길이 출력
    def __len__(self):
        return (len(self.labels))

In [17]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [18]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 패딩
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

# 데이터 확인 -> array1은 시퀀스, array2는 길이와 타입, array3은 attention mask 시퀀스(1로 패딩된 값들은 연산할 필요가 없기 때문에, 연산을 하지 않아도 됨 -> 이걸 알려주는 데이터가 바로 얘)
data_train[0]

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


(array([   2, 3257, 2704, 6733, 7101, 5050, 7088, 2740, 7867, 1458, 4207,
        7782, 6601, 6553, 1947, 6668,  517,   54,    3,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(19, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [19]:
# torch 형식의 dataset 생성
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [20]:
# KoBERT 학습모델
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,   # 클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [21]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio) # 초기에 성능을 끌어올리는 것 = 운동할 때 워밍업 느낌

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0] # 맞은 비율
    return train_acc
    
train_dataloader

In [22]:
# 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/804 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.4394432306289673 train acc 0.203125
epoch 1 batch id 201 loss 1.1693743467330933 train acc 0.39637748756218905
epoch 1 batch id 401 loss 0.7010747194290161 train acc 0.5177680798004988
epoch 1 batch id 601 loss 0.6024736166000366 train acc 0.579450915141431
epoch 1 batch id 801 loss 0.5734092593193054 train acc 0.6155391697877652
epoch 1 train acc 0.6157708578816444


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

  0%|          | 0/268 [00:00<?, ?it/s]

epoch 3 test acc 0.7391474546908315


  0%|          | 0/804 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.523074746131897 train acc 0.828125


KeyboardInterrupt: ignored

In [ ]:
'''
import matplotlib.pyplot as plt
plt.plot(np.arange(0.0, 7.0, 1.0), train_acc_list, np.arange(0.0, 7.0, 1.0), test_acc_list, 'r-')
'''

# 모델 평가

In [23]:
import numpy as np
import matplotlib.pyplot as plt
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [33]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

label_softmax = []
# 새로운 문장 테스트
def predict(predict_sentence):
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      #print("token_ids : ", token_ids)
      segment_ids = segment_ids.long().to(device)
      #print("segment_ids : ", segment_ids)

      valid_length= valid_length
      #print("valid_length : ", valid_length)
      label = label.long().to(device)
      #print("label : ", label)
      out = model(token_ids, valid_length, segment_ids)

      #print("out : ", out)

      test_eval=[]
      for i in out:
          logits=i
          logits = logits.detach().cpu().numpy()
          logits = softmax(logits)
          print("softmax : ", logits)
          label_softmax = logits

          if np.argmax(logits) == 0:
                test_eval.append("기쁨, 행복")
          elif np.argmax(logits) == 1:
                test_eval.append("분노, 불안, 당황, 놀람")
          elif np.argmax(logits) == 2:
                test_eval.append("슬픔, 상처")
          elif np.argmax(logits) == 3:
                test_eval.append("중립")

                # 중립 삭제 코드
        
          


      print(label_softmax)
      print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
      #print(str(label_softmax[0]) + "기쁨 " + str(label_softmax[1]) + "불안 " + str(label_softmax[2]) + "당황 " + str(label_softmax[3]) + "슬픔 " + str(label_softmax[4]) + "분노 " + str(label_softmax[5]) + "상처 ")

      return label_softmax

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


softmax :  [0.00361323 0.03129457 0.96065944 0.0044327 ]
[0.00361323 0.03129457 0.96065944 0.0044327 ]
>> 입력하신 내용에서 슬픔, 상처 느껴집니다.


softmax :  [0.00317183 0.0297137  0.9643537  0.00276071]
[0.00317183 0.0297137  0.9643537  0.00276071]
>> 입력하신 내용에서 슬픔, 상처 느껴집니다.




In [28]:
!pip install kss

     |████████████████████████████████| 42.4 MB 1.4 MB/s 
     |████████████████████████████████| 170 kB 67.6 MB/s 
  Created wheel for kss: filename=kss-3.3.1.1-py3-none-any.whl size=42449241 sha256=27820dcdd29af270cb6772447bc327c4f8ea5e37a753636b35fa2f71b506ac67
  Stored in directory: /root/.cache/pip/wheels/6e/9d/1d/52871154eff5273abb86b96f4f984c1cd67c5bde64239b060a
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169313 sha256=962c08bbfbea0e2c85e111772ca95852a4de6da24035ad427ae78073cb215ee6
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built kss emoji


In [29]:
from kss import split_sentences

여기 경기도 군포인데해겁나 쨍쨍한데 눈부셔	

In [60]:
sentence = input("하고싶은 말을 입력해주세요 : ")

softmax_list = []
for sent in split_sentences(sentence):
    print(sent)
    softmax_temp = predict(sent)
    softmax_list.append(softmax_temp)

    print(" ")
    #predict(sentence)
    

하고싶은 말을 입력해주세요 : 오늘 이별을 할 것 같은 느낌이 있었다. 아침에 애인을 만났는데 표정이 좋지 않았었기 때문이다. 불길한 예감은 역시나 이별을 통보 받았고, 난 헤어졌다. 여기 경기도 군포인데해겁나 쨍쨍한데 눈부셔. 지금 미치도록 괴롭다.
오늘 이별을 할 것 같은 느낌이 있었다.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


softmax :  [0.00870455 0.01734697 0.9699377  0.00401077]
[0.00870455 0.01734697 0.9699377  0.00401077]
>> 입력하신 내용에서 슬픔, 상처 느껴집니다.
 
아침에 애인을 만났는데 표정이 좋지 않았었기 때문이다.
softmax :  [0.00715858 0.43782303 0.5531292  0.00188923]
[0.00715858 0.43782303 0.5531292  0.00188923]
>> 입력하신 내용에서 슬픔, 상처 느껴집니다.
 
불길한 예감은 역시나 이별을 통보 받았고, 난 헤어졌다.
softmax :  [0.00544007 0.02280559 0.96778697 0.00396738]
[0.00544007 0.02280559 0.96778697 0.00396738]
>> 입력하신 내용에서 슬픔, 상처 느껴집니다.
 
여기 경기도 군포인데해겁나 쨍쨍한데 눈부셔.
softmax :  [0.04554021 0.13384253 0.10866887 0.7119484 ]
[0.04554021 0.13384253 0.10866887 0.7119484 ]
>> 입력하신 내용에서 중립 느껴집니다.
 
지금 미치도록 괴롭다.
softmax :  [0.00221924 0.04972576 0.94594383 0.00211122]
[0.00221924 0.04972576 0.94594383 0.00211122]
>> 입력하신 내용에서 슬픔, 상처 느껴집니다.
 


In [40]:
softmax_list

[array([0.00948551, 0.0180901 , 0.9685145 , 0.00390988], dtype=float32),
 array([0.00715858, 0.43782303, 0.5531292 , 0.00188923], dtype=float32),
 array([0.00544007, 0.02280559, 0.96778697, 0.00396738], dtype=float32),
 array([0.00340099, 0.03622873, 0.9571088 , 0.00326145], dtype=float32)]

In [44]:
softmax_1_sum = softmax_list[0][0] + softmax_list[1][0] + softmax_list[2][0] + softmax_list[3][0]
softmax_2_sum = softmax_list[0][1] + softmax_list[1][1] + softmax_list[2][1] + softmax_list[3][1]
softmax_3_sum = softmax_list[0][2] + softmax_list[1][2] + softmax_list[2][2] + softmax_list[3][2]
softmax_4_sum = softmax_list[0][3] + softmax_list[1][3] + softmax_list[2][3] + softmax_list[3][3]

In [46]:
softmax_sum_list = []
softmax_sum_list.append(softmax_1_sum)
softmax_sum_list.append(softmax_2_sum)
softmax_sum_list.append(softmax_3_sum)
softmax_sum_list.append(softmax_4_sum)
softmax_sum_list

[0.025485152, 0.5149475, 3.4465394, 0.013027941]

In [47]:
softmax(softmax_sum_list)

array([0.02922206, 0.04767401, 0.89424366, 0.0288603 ], dtype=float32)

# 모델 저장

In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/nlp/model_state_dict.pt')

In [ ]:
torch.save(model, '/content/drive/My Drive/nlp/model.pt')

In [ ]:
'''
저 이번주에 결혼해요"

먼저 낚을 생각은 많았지만 그래도 본의에 맞게 살짝쿵 놀라게 해드린 점 사과드린다.

리스본 글쓰기 클럽에서 읽다가 너무 재미져서 허락을 맡고 가져와봤다. 혹시 모르니 이 글은 48시간 뒤에 폭파될 예정.

나는 결혼식이 싫다. 기계적으로 친목회비를 걷어 '00선생님 아들내미 결혼식 축 00만원 지급' 이라는 메신저가 올 때는 몰랐었다. 앞으로 내게 쌓일 (주로) 모바일 청첩장의 축의금은 내가 정해야한다는 사실을. 폐지된 개콘 코너인 애(매한 것) 정(해드리는)남(자) 코너에서 축의금 액수 딱 정해드립니다! 보고 웃는 방청객들이 이해가 가지 않았던 게 엊그제 같은데 벌써 친구 결혼식을 가야 할 나이가 되어버렸다. 이제는 누가 좀 정해줬으면 좋겠다.(개콘은 왜 폐지되서...엉엉)

나는 축의금이 이렇게 사람을 쪼잔하게 만드는 건지 몰랐다. 그간 그 사람과 함께해온 추억과 세월과 정을 숫자로 환산할 수 있다니, 이과들은 물러가라. 문과로서는 도저히 용납할 수 없다. 앞서 말한 것들이 축의금 액수로 환산되어 명부에 적히는 순간, 내 우정도 숫자로 환산될 것만 같은 느낌이다.

만수르라면 고민 없이 신혼부부가 살기좋은 강남 역세권 18평짜리 아파트라도 내놓겠지만 대한민국 20대 사회초년생은 손을 덜덜 떨며 입에 풀칠하기 바쁜 피같은 월급을 쪼개 내야 한다. 그마저도 별 하나에 추억과 별 하나에 이름과 별 하나에 어머니..!!! 하며 너와 나의 우정에 얼만큼의 축의금이 적당한지 밤하늘의 별과 함께 헤아려야한다.

이제 겨우 스타트를 끊은 친구 결혼식이다. 앞으로도 수많은 이들의 결혼식에 가기 위해 축의금 액수를 고민하고, 하객룩을 고민하고 그간의 인간관계에 대해 고민하게 될 생각을 하면 벌써부터 질린다.

물론 이 글을 쓰고도 나는 브라이덜 샤워에 가서 흰 옷을 입고 신부를 빛내주기 위해 이 한 몸 다 바쳐 노력 중일 테다.
